# General Traits of Good Code

### Diseño por contrato:

* Precondiciones: son todas las condiciones que se checkean antes de que la función sea ejecutada, esto ayuda a detectar fácilmente los errores que vienen de parte del cliente, es decir en la llamada, caller, API etc

* Postcondiciones: son todas aquellas pruebas que se hacen luego de que se ejecutó la función, con esto podemos detectar si el error generado es de parte de nuestra función                

* Invariantes: partes del código que se mantienen constantes durante la ejecución

* Efectos-secundarios

Lo ideal es tener las precondiciones, las post condiciones y el core de la función separados
    

### Programación defensiva:

A diferencia de la programación por contrato, en lugar de tener todas las condiciones que debe cumplir una función, y si no se cumplen que falle nuestro código, está más enfocada a que todas las partes del código sean capaces de protegerse a sí mismas contra cualquier entrada inválida, Las ideas principales sobre el tema de la programación defensiva son cómo manejar los errores para escenarios que podríamos esperar que ocurran, y cómo lidiar con errores que nunca deberían ocurrir (cuando ocurren condiciones imposibles).

## Manejo de errores:
Existen diferentes enfoques para el manejo de errores:
* Value substitution: En algunos escenarios, cuando hay un error y existe el riesgo de que el software produzca valor incorrecto o falla por completo, podríamos ser capaces de reemplazar el resultado con otro más seguro valor. Tiene que ser elegida cuidadosamente para casos donde el valor sustituido es en realidad una opción segura. Esto podría no ser aceptable para algunos tipos de software. Si la aplicación es crítica, o la los datos que se manejan son demasiado confidenciales, esta no es una opción, ya que no podemos darnos el lujo de proporcionar usuarios (u otras partes de la aplicación) con resultados erróneos, entonces se deja que mejor explote.

- Error logging: 

- Exception handling: En este enfoque es donde se usan las excepciones **pero es muy importante resaltar que las excepeciones se usan es precisamente para anunciar una situacion atípica, o excepcional, no alterando el flujo o reglas del programa, si el código trata de usar las excepciones como un flujo natural del código, va a ser mucho más dificil de leer, es muy importante tener esto en cuenta**   Si una función tiene demasiadas excepciones es un sintoma de que la función tiene demasiadas responsabilidades y debería pensarse en dividirse en funciones más pequeñas

#### Manejar las excepciones en nivel correcto de abstracción:
Ej:

In [2]:
import logging
import time

logger = logging.getLogger(__name__)

class Connector:
    """Abstract the connection to a database."""

    def connect(self):
        """Connect to a data source."""
        return self

    @staticmethod
    def send(data):
        return data

class Event:
    def __init__(self, payload):
        self._payload = payload

    def decode(self):
        return f"decoded {self._payload}"

class DataTransport:
    """An example of an object badly handling exceptions of different levels."""

    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None

    def deliver_event(self, event):
        try:
            self.connect()
            data = event.decode()
            self.send(data)
        except ConnectionError as e:
            logger.info("connection error detected: %s", e)
            raise
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

    def connect(self):
        for _ in range(self.retry_n_times):
            try:
                self.connection = self._connector.connect()
            except ConnectionError as e:
                logger.info(
                    "%s: attempting new connection in %is",
                    e,
                    self.retry_threshold,
                )
                time.sleep(self.retry_threshold)
            else:
                return self.connection
        raise ConnectionError(
            f"Couldn't connect after {self.retry_n_times} times"
        )

    def send(self, data):
        return self.connection.send(data)

En el método deliver_event(), hay dos tipos diferentes de excepciones, podríamos intuir de que las responsabilidades de la función podríar dividirse así que `ConnectionError` debería ser manipulado dentro del método `connect()`, y de igual forma `ValueError` corresponde al método `decode`, de esta forma el método ya no se tiene que preocupar explicitamente por estas excepciones sino directamente los método invocados

#### No mostrar los tracebacks

Esta es una consideración de seguridad, Al mismo tiempo, queremos incluir tanto detalle lo más posible para nosotros mismos; definitivamente no queremos que nada de esto sea visible para usuarios. Si elige permitir que se propaguen las excepciones, asegúrese de no revelar ninguna información confidencial

#### No dejar excepciones vacías:

Es una de las prácticas más malas, a pesar de que es bueno anticiparse y defender nuestro codigo de errores, ser demasiado defensivo podría llevar a errores peores, el problema con una excepción vacía es que no va a fallar, entonces los errores van a pasar desapercibidos

#### Incluir la excepción original:

Como parte de nuestra lógica de manejo de errores, podríamos decidir plantear una diferente y tal vez incluso cambiar su mensaje. Si ese es el caso, se recomienda incluir la excepción original que condujo a eso, se recomienda usar la sintaxis `raise <e> from <original_exception> `, esto siempre se debe de usar cuando se cambia el tipo de la expceción


### Asserts

Los asserts deben usarse para situaciones que nunca deberían suceder, por lo que la expresión en el assert tiene que significar una condición imposible. Si ocurriera esta condición,significa que hay un defecto en el software. En contraste con el enfoque de manejo de errores, aquí existe (o no debería existir) la posibilidad de continuando el programa. Si se produce dicho error, el programa debe detenerse. Tiene sentido detenga el programa porque, como se comentó anteriormente, estamos en presencia de un defecto.

**Importante: No atrapar los errores de los assert en las excepciones**

